In [5]:
!pip install source-coop
!pip install ipywidgets

Requirement already satisfied: pure-eval in c:\pythonenv\spatial\lib\site-packages (from 
stack-data->ipython>=6.1.0->ipywidgets) (0.2.2)

Requirement already satisfied: six>=1.12.0 in c:\pythonenv\spatial\lib\site-packages (from 
asttokens>=2.1.0->stack-data->ipython>=6.1.0->ipywidgets) (1.16.0)

Installing collected packages: widgetsnbextension, jupyterlab-widgets, ipywidgets

Successfully installed ipywidgets-8.1.5 jupyterlab-widgets-3.0.13 widgetsnbextension-4.0.13

es_cm.parquet ━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━    10.4%   • 0.2/2.3 GB    • 3.5 MB/s • 0:09:51
   es_cm.pmtiles ━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━    23.0%   • 0.3/1.1 GB    • 5.4 MB/s • 0:02:42
 collection.json ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━    100.0%  • 2.3/2.3 kB    • ?        • 0:00:00
       README.md ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━    100.0%  • 1.9/1.9 kB    • ?        • 0:00:00
     LICENSE.txt ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━    100.0%  • 274/274 bytes • ?        • 0:00:00

In [1]:
from source_coop import SourceCoopClient

def list_featured_repositories():
    client = SourceCoopClient()

    # Get featured repositories
    featured = client.api.get_repositories(featured=True)

    if featured and featured.get('repositories'):
        print(f"Found {len(featured['repositories'])} featured repositories:")

        for repo in featured['repositories']:
            title = repo.get('meta', {}).get('title', 'Untitled')
            account = repo.get('account_id', 'Unknown')

            print(f"- {title} by {account}")
    else:
        print("No featured repositories found.")

if __name__ == "__main__":
    list_featured_repositories()

Found 9 featured repositories:
- Overture Maps - Fused-partitioned by fused
- Land, carbon and biodiversity data for supply chain impact calculation by vizzuality
- National Wetlands Inventory by giswqs
- Google-Microsoft-OSM Open Buildings - combined by VIDA by vida
- Energy Performance Certificate Ratings (Domestic) - England and Wales by addresscloud
- RapidAI4EO by planet
- Fields of The World (FTW) by kerner-lab
- Multi-Temporal Crop Classification with HLS Imagery across CONUS  by clarkcga
- Archive of data.gov by harvard-lil


#### Search Repos
This code snippet will allow you to search for repos by tags. Here we are using the tag "Sentinel"

In [2]:
from source_coop import SourceCoopClient

def get_prefix(data_dict):
      outer_key = next(iter(data_dict))
      prefix = data_dict[outer_key]['prefix']
      return prefix

def search_repositories(query, limit=10):
    client = SourceCoopClient()

    # Search for repositories
    results = client.api.get_repositories(search=query, limit=limit)

    if results and results.get('repositories'):
        print(f"Found {len(results['repositories'])} repositories matching '{query}':")

        for repo in results['repositories']:
            repo_prefix = get_prefix(repo.get('data', 'Unknown').get('mirrors','Unknown'))
            title = repo.get('meta', {}).get('title', 'Untitled')
            account = repo.get('account_id', 'Unknown')
            tags = ", ".join(repo.get('meta', {}).get('tags', []))

            print(f"- {title} by {account}")
            print(f"Repo Url :https://source.coop/repositories/{repo_prefix}")
            if tags:
                print(f"  Tags: {tags}")
    else:
        print(f"No repositories found matching '{query}'.")

if __name__ == "__main__":
    search_repositories("fiboa")

Found 10 repositories matching 'fiboa':
- Open Administrative Boundaries by cholmes
Repo Url :https://source.coop/repositories/cholmes/admin-boundaries/
  Tags: geoparquet, admin, countries, overture
- Spain Navarra Crop fields by fiboa
Repo Url :https://source.coop/repositories/fiboa/es-nc/
  Tags: navarra, spain, sigpac, fiboa, field boundaries, geoparquet, vector, pmtiles
- Spain Castilla-La Mancha Crop fields by fiboa
Repo Url :https://source.coop/repositories/fiboa/es-cm/
  Tags: castillalamancha, spain, sigpac, fiboa, field boundaries, geoparquet, vector, pmtiles
- Gallicia SIXPAC Crop Fields by fiboa
Repo Url :https://source.coop/repositories/fiboa/es-ga/
  Tags: galicia, spain, sigpac, sixpac, fiboa, field boundaries, geoparquet, vector, pmtiles
- Comunidad de Madrid Crop Fields by fiboa
Repo Url :https://source.coop/repositories/fiboa/es-md/
  Tags: madrid, spain, sigpac, fiboa, field boundaries, geoparquet, vector, pmtiles
- Spain Cantabria Crop fields by fiboa
Repo Url :http

In [6]:
import asyncio
import nest_asyncio
from source_coop import SourceCoopClient
from source_coop.s3 import SourceCoopS3
from source_coop.commands.download import download_s3_objects_async

# Apply nest_asyncio patch
nest_asyncio.apply()

async def download_repository_async(repository_url, output_dir=None, file_type=None):
    client = SourceCoopClient()

    if repository_url.startswith("http"):
        s3_url = SourceCoopS3.convert_repo_url_to_s3_url(repository_url)
        if not s3_url:
            print("Failed to convert repository URL to S3 URL.")
            return False
    else:
        s3_url = repository_url

    print(f"Using S3 URL: {s3_url}")

    if not output_dir:
        _, prefix = SourceCoopS3.parse_s3_url(s3_url)
        repo_name = prefix.strip('/').split('/')[-1] if prefix else "download"
        output_dir = f"./source-coop-{repo_name}"

    print(f"Using output directory: {output_dir}")

    print("Listing objects...")
    objects, summary = client.s3.list_objects_with_summary(s3_url, file_type=file_type)

    if not objects:
        print("No objects found to download.")
        return False

    print(f"\nFound {summary['total_files']} files ({summary['total_size_human']})")

    successful = await download_s3_objects_async(
        objects, output_dir, max_concurrent=10, multipart_count=8, quiet=False
    )

    print(f"\nDownload complete. {successful} files downloaded successfully.")
    return successful > 0

def download_repository_colab(repository_url, output_dir=None, file_type=None):
    loop = asyncio.get_event_loop()
    return loop.run_until_complete(download_repository_async(repository_url, output_dir, file_type))

# Example Usage:
download_repository_colab("https://source.coop/repositories/fiboa/es-cm/")


es_cm.parquet ━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━    18.6%   • 0.4/2.3 GB    • 3.4 MB/s • 0:09:12
   es_cm.pmtiles ━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━    37.8%   • 0.4/1.1 GB    • 3.0 MB/s • 0:03:59
 collection.json ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━    100.0%  • 2.3/2.3 kB    • ?        • 0:00:00
       README.md ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━    100.0%  • 1.9/1.9 kB    • ?        • 0:00:00
     LICENSE.txt ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━    100.0%  • 274/274 bytes • ?        • 0:00:00